In [3]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
model_path = "../chatglm3-6b-base"
device = 'cpu'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).float()
# model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).half().cuda()

Loading checkpoint shards: 100%|██████████| 7/7 [00:18<00:00,  2.63s/it]


## 重写ChatGLM3-6b-Base的stream_generate()函数来嵌入Prompt Lookup Decoding函数

In [17]:
import math
import copy
import warnings
import re
import sys
import time
sys.path.append('../chatglm3-6b-base')

import torch
import torch.utils.checkpoint
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, LayerNorm
from torch.nn import CrossEntropyLoss, LayerNorm, MSELoss, BCEWithLogitsLoss
from torch.nn.utils import skip_init
from typing import Optional, Tuple, Union, List, Callable, Dict, Any
from copy import deepcopy

from transformers.modeling_outputs import (
    BaseModelOutputWithPast,
    CausalLMOutputWithPast,
    SequenceClassifierOutputWithPast,
)
from transformers.modeling_utils import PreTrainedModel
from transformers.utils import logging
from transformers.generation.logits_process import LogitsProcessor
from transformers.generation.utils import LogitsProcessorList, StoppingCriteriaList, GenerationConfig, ModelOutput, _crop_past_key_values

from configuration_chatglm import ChatGLMConfig

### modify for Prompt Lookup Decoding
@torch.no_grad()
def find_candidate_pred_tokens(input_ids, max_ngram_size=3, num_pred_tokens=10):
    input_length = input_ids.size(1)

    # Ensure max_ngram_size and num_pred_tokens are valid
    if max_ngram_size <= 0 or num_pred_tokens <= 0 or max_ngram_size > input_length:
        raise ValueError("Invalid max_ngram_size or num_pred_tokens")

    for ngram_size in range(max_ngram_size, 0, -1):
        # Extract the last n tokens as our search ngram
        ngram = input_ids[0, -ngram_size:].tolist()

        # Create sliding windows of size ngram_size
        windows = input_ids.unfold(dimension=1, size=ngram_size, step=1)

        # Convert ngram to a tensor for comparison
        ngram_tensor = torch.tensor(ngram, device=input_ids.device).unsqueeze(0)

        # Find where the windows match the ngram
        matches = (windows == ngram_tensor).all(dim=2)

        # Get the indices of matches
        match_indices = matches.nonzero(as_tuple=True)[1]

        # Iterate through match indices to find a valid continuation
        for idx in match_indices:
            start_idx = idx + ngram_size
            end_idx = start_idx + num_pred_tokens
            # Ensure we don't go beyond the length of input_ids and avoid self-match
            if end_idx <= input_length and start_idx < input_length - ngram_size:
                return input_ids[0, start_idx:end_idx]

    # If no match is found, return an empty tensor
    return torch.tensor([], dtype=torch.long, device=input_ids.device)


@torch.inference_mode()
def stream_generate_assisted_by_prompt_lookup_decoding(
        self,
        input_ids,
        generation_config: Optional[GenerationConfig] = None,
        logits_processor: Optional[LogitsProcessorList] = None,
        stopping_criteria: Optional[StoppingCriteriaList] = None,
        prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None,
        return_past_key_values=False,
        draft_matching_window_size = 3,
        draft_num_candidate_tokens = 10,
        **kwargs,
):
    """
    重新ChatGLM3-6b-base的stream_generate()函数
    """
    batch_size, input_ids_seq_length = input_ids.shape[0], input_ids.shape[-1]

    if generation_config is None:
        generation_config = self.generation_config
    generation_config = copy.deepcopy(generation_config)
    model_kwargs = generation_config.update(**kwargs)
    model_kwargs["use_cache"] = generation_config.use_cache
    bos_token_id, eos_token_id = generation_config.bos_token_id, generation_config.eos_token_id

    if isinstance(eos_token_id, int):
        eos_token_id = [eos_token_id]
    eos_token_id_tensor = torch.tensor(eos_token_id).to(input_ids.device) if eos_token_id is not None else None

    has_default_max_length = kwargs.get("max_length") is None and generation_config.max_length is not None
    if has_default_max_length and generation_config.max_new_tokens is None:
        warnings.warn(
            f"Using `max_length`'s default ({generation_config.max_length}) to control the generation length. "
            "This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we"
            " recommend using `max_new_tokens` to control the maximum length of the generation.",
            UserWarning,
        )
    elif generation_config.max_new_tokens is not None:
        generation_config.max_length = generation_config.max_new_tokens + input_ids_seq_length
        if not has_default_max_length:
            # logger.warn(
            #     f"Both `max_new_tokens` (={generation_config.max_new_tokens}) and `max_length`(="
            #     f"{generation_config.max_length}) seem to have been set. `max_new_tokens` will take precedence. "
            #     "Please refer to the documentation for more information. "
            #     "(https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)",
            #     UserWarning,
            # )
            print(
                f"Both `max_new_tokens` (={generation_config.max_new_tokens}) and `max_length`(="
                f"{generation_config.max_length}) seem to have been set. `max_new_tokens` will take precedence. "
                "Please refer to the documentation for more information. "
                "(https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)",
                UserWarning,
            )
            

    if input_ids_seq_length >= generation_config.max_length:
        input_ids_string = "decoder_input_ids" if self.config.is_encoder_decoder else "input_ids"
        # logger.warning(
        #     f"Input length of {input_ids_string} is {input_ids_seq_length}, but `max_length` is set to"
        #     f" {generation_config.max_length}. This can lead to unexpected behavior. You should consider"
        #     " increasing `max_new_tokens`."
        # )
        print(
            f"Input length of {input_ids_string} is {input_ids_seq_length}, but `max_length` is set to"
            f" {generation_config.max_length}. This can lead to unexpected behavior. You should consider"
            " increasing `max_new_tokens`."
        )

    # 2. Set generation parameters if not already defined
    logits_processor = logits_processor if logits_processor is not None else LogitsProcessorList()
    stopping_criteria = stopping_criteria if stopping_criteria is not None else StoppingCriteriaList()

    logits_processor = self._get_logits_processor(
        generation_config=generation_config,
        input_ids_seq_length=input_ids_seq_length,
        encoder_input_ids=input_ids,
        prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
        logits_processor=logits_processor,
    )

    stopping_criteria = self._get_stopping_criteria(
        generation_config=generation_config, stopping_criteria=stopping_criteria
    )
    max_len = stopping_criteria[0].max_length

    logits_warper = self._get_logits_warper(generation_config)

    unfinished_sequences = input_ids.new(input_ids.shape[0]).fill_(1)
    scores = None
    while True:
        
        cur_len = input_ids.shape[-1]
        # print(f'cur_len: {cur_len}')

        # 1. 从已有的input_ids中查找candidate_pred_tokens
        # start_time = time.time()
        candidate_pred_tokens = find_candidate_pred_tokens(input_ids, draft_matching_window_size, draft_num_candidate_tokens)
        # print("Lookup耗时=", (time.time() - start_time))

        if len(candidate_pred_tokens) == 0:
            candidate_pred_tokens = torch.tensor([100], device=input_ids.device).unsqueeze(0)
        else:
            candidate_pred_tokens = candidate_pred_tokens.unsqueeze(0)

        # 2. 将候选token与input_ids做拼接
        candidate_input_ids = torch.cat((input_ids, candidate_pred_tokens), dim=1)
        
        candidate_length = candidate_input_ids.shape[1] - input_ids.shape[1]
        # print('candidate_length: ', candidate_length)
       

        # 3. 构造模型输入
        model_inputs = {"return_last_logit": False}
        if model_kwargs["is_first_forward"]:
            # 初次运行，输入所有input_ids
            model_inputs["input_ids"] = candidate_input_ids
            model_inputs["position_ids"] = torch.unsqueeze(torch.range(0, candidate_input_ids.shape[1]-1, 1, dtype=torch.int64), dim=0)
            model_inputs["attention_mask"] = model_kwargs["attention_mask"] 
            model_inputs["past_key_values"] = None
        else:
            # 非初次运行，仅输入上一次推理的最后一个token和所有候选tokens
            model_inputs["input_ids"] = candidate_input_ids[:, min(-candidate_length-1, -1):]
            model_inputs["position_ids"] = torch.unsqueeze(torch.range(0, candidate_input_ids.shape[1]-1, 1, dtype=torch.int64), dim=0)[..., min(-candidate_length-1, -1):]
            model_inputs["attention_mask"] = None
            model_inputs["past_key_values"] = model_kwargs["past_key_values"]

        # print(f'inputs_ids: {model_inputs["input_ids"].size()}')
        # print(f'position_ids: {model_inputs["position_ids"].size()}')
        # if model_inputs["past_key_values"] is not None:
        #     print(f'past_key_values: {len(model_inputs["past_key_values"])}')
        #     print(f'past_key_values: {len(model_inputs["past_key_values"][0])}')
        #     print(f'past_key_values: {model_inputs["past_key_values"][0][0].size()}')
        # print('model_inputs: ', model_inputs)

        # start_time = time.time()
        # forward pass to get next token
        outputs = self(
            **model_inputs,
            return_dict=True,
            output_attentions=False,
            output_hidden_states=False
        )
        # print("模型推理耗时=", (time.time() - start_time))

        # print('outputs.logits', outputs.logits.size())

        if model_kwargs["is_first_forward"] :
            new_logits = outputs.logits[:, -candidate_length - 1 :]  # excludes the input prompt if present
        else:
            new_logits = outputs.logits
        # print(f'new_logits: {new_logits.size()}')
        selected_tokens = new_logits.argmax(dim=-1)
        # print(f'selected_tokens: {selected_tokens.size()}')
        # print('*******-', tokenizer.decode(selected_tokens[0].tolist()))
        candidate_new_tokens = candidate_input_ids[:, -candidate_length:]
        # print(f'candidate_new_tokens: {candidate_new_tokens.size()}')
        # print(tokenizer.decode(candidate_new_tokens[0].tolist()))
        
        # 4.查找llm输出与候选token匹配的项
        #   查找策略：从第一个候选token开始逐一检查，一旦遇到不相互匹配的token，经保留最后一个不匹配的项，其它都舍弃
        n_matches = ((~(candidate_new_tokens == selected_tokens[:, :-1])).cumsum(dim=-1) < 1).sum()

        n_matches = min(n_matches, max_len - cur_len - 1)

        valid_tokens = selected_tokens[:, : n_matches + 1]
        input_ids = torch.cat((input_ids, valid_tokens), dim=-1)
        last_token = valid_tokens[: , -1]

        new_cur_len = input_ids.shape[-1]

        new_cache_size = new_cur_len - 1
        
        # 5.根据第四步中的匹配结果（多少个候选token与LLM输出一致），抽取部分kv_cache
        kv_caches = []
        for idx in range(len(outputs.past_key_values)):
            kv_caches.append((outputs.past_key_values[idx][0][:new_cache_size, :,:,:],
                              outputs.past_key_values[idx][1][:new_cache_size, :,:,:]))
        
        model_kwargs["past_key_values"] = tuple(kv_caches)
        model_kwargs["is_first_forward"] = False
              
        
        unfinished_sequences = unfinished_sequences.mul(
            last_token.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)
        )
        if return_past_key_values:
            yield input_ids, outputs.past_key_values
        else:
            yield input_ids
        # stop when each sentence is finished, or if we exceed the maximum length
        if unfinished_sequences.max() == 0 or stopping_criteria(input_ids, scores):
            break

model.stream_generate_assisted_by_prompt_lookup_decoding = stream_generate_assisted_by_prompt_lookup_decoding.__get__(model, type(model))

In [18]:
prompts = ["""<system>你是一个AI助手，旨在提供帮助、诚实和无害的对话服务。<user>风吹人，你能解释一下吗<assistant>风吹人是通过空调摄像头，计算出人体实时位置，仅对该区域进行送风。送风区域可以跟随人体走动而随时变化，人动，风也动。</s><user>风吹人功能在空调上要怎么进行操作<assistant>空调的风吹人功能是通过空调摄像头计算出人体的实时位置，然后仅对该区域进行送风。当人体移动时，送风区域也会随之改变，实现人动风也动的效果。</s><user>风避人的原理是什么，怎么做到的<assistant>风避人通过空调摄像头计算出人体实时位置，让风避开人体位置。避风区域能随人体走动而变化，人在哪，避风区域就在哪。</s><user>可供参考的知识库有：
问题:智慧知冷暖是什么的答案是智慧知冷暖是在制冷和制热模式下，红外传感器在有效检测范围内检测房间空气温度和人体表温度，空调结合房间空气湿度、空调送风状态等参数，计算人体冷热感。空调根据冷热感，自动调整运行参数，不用人手动遥控空调，让房间空气温度达到让人体感到舒适的状态。
问题:智慧知冷暖的运行参数是如何自动调整的的答案是智慧知冷暖通过红外传感器检测房间空气温度和人体表温度，结合房间空气湿度、空调送风状态等参数，计算人体冷热感。然后根据这些数据，自动调整运行参数，使得房间空气温度达到让人体感到舒适的状态，无需人工手动调整。
知识点:风避人是通过空调摄像头计算出人体实时位置，让风避开人体位置，避风区域可以根据人体的走动而变化。
知识点:空调的风避人功能是通过摄像头计算出人体实时位置，让风避开人体。避风区域可以随着人体走动而变化，实现人在哪，避风区域就在哪的效果。
知识点:风避人是通过空调摄像头，计算出人体实时位置，让风避开人体位置。避风区域可以跟随人体走动而随时变化，人在哪里，避风区域就在哪里。
知识点:风避人功能通过空调摄像头计算出人体实时位置，避风区域会跟随人体走动而变化，即人在哪里，避风区域就在哪里。
知识点:风避人通过空调摄像头计算出人体实时位置，然后让风避开人体位置。避风区域能随人体走动而变化，人在哪，避风区域就在哪。
问题:智慧知冷暖的运行方式是怎样的的答案是智慧知冷暖通过红外传感器检测房间空气温度和人体表温度，结合房间空气湿度和空调送风状态等参数，计算人体冷热感。然后根据这个冷热感，自动调整空调运行参数，使得房间温度达到人体感到舒适的状态，无需人手动遥控。
问题:智慧知冷暖是怎么运作的的答案是智慧知冷暖通过红外传感器检测房间空气温度和人体表温度，结合房间空气湿度和空调送风状态等参数，计算人体冷热感。然后根据这个冷热感，自动调整空调运行参数，使得房间温度达到人体感到舒适的状态，无需人手动遥控。
根据这些搜索结果回答关于空调的问题。如果知识库中没有答案，请拒绝回答。如果有答案，请回答不超过20个字的文本。
问题：智慧知冷暖是如何运行的的最简短答案是："""
]

### 调用使用PLD的生成函数：（每一行表示一次模型推理结果）

In [20]:
for p in prompts:
    inputs = tokenizer(p, return_tensors="pt").to(device)
    inputs['max_length'] = 2000
    inputs['is_first_forward'] = True
    stream_generate_pld = model.stream_generate_assisted_by_prompt_lookup_decoding(**inputs)
    cur_len = 0
    s_time = time.time()
    for i in stream_generate_pld:
        new_words = tokenizer.decode(i[0][cur_len:].tolist())
        cur_len = len(i[0])
        print(new_words)
        # print(tokenizer.decode(i[0].tolist()))
    print(f'共耗时={(time.time() - s_time)} ')
    print("*************************************************************************************")

/tmp/ipykernel_3332676/1086294788.py:185: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  model_inputs["position_ids"] = torch.unsqueeze(torch.range(0, candidate_input_ids.shape[1]-1, 1, dtype=torch.int64), dim=0)


[gMASK]sop <system>你是一个AI助手，旨在提供帮助、诚实和无害的对话服务。<user>风吹人，你能解释一下吗<assistant>风吹人是通过空调摄像头，计算出人体实时位置，仅对该区域进行送风。送风区域可以跟随人体走动而随时变化，人动，风也动。</s><user>风吹人功能在空调上要怎么进行操作<assistant>空调的风吹人功能是通过空调摄像头计算出人体的实时位置，然后仅对该区域进行送风。当人体移动时，送风区域也会随之改变，实现人动风也动的效果。</s><user>风避人的原理是什么，怎么做到的<assistant>风避人通过空调摄像头计算出人体实时位置，让风避开人体位置。避风区域能随人体走动而变化，人在哪，避风区域就在哪。</s><user>可供参考的知识库有：
问题:智慧知冷暖是什么的答案是智慧知冷暖是在制冷和制热模式下，红外传感器在有效检测范围内检测房间空气温度和人体表温度，空调结合房间空气湿度、空调送风状态等参数，计算人体冷热感。空调根据冷热感，自动调整运行参数，不用人手动遥控空调，让房间空气温度达到让人体感到舒适的状态。
问题:智慧知冷暖的运行参数是如何自动调整的的答案是智慧知冷暖通过红外传感器检测房间空气温度和人体表温度，结合房间空气湿度、空调送风状态等参数，计算人体冷热感。然后根据这些数据，自动调整运行参数，使得房间空气温度达到让人体感到舒适的状态，无需人工手动调整。
知识点:风避人是通过空调摄像头计算出人体实时位置，让风避开人体位置，避风区域可以根据人体的走动而变化。
知识点:空调的风避人功能是通过摄像头计算出人体实时位置，让风避开人体。避风区域可以随着人体走动而变化，实现人在哪，避风区域就在哪的效果。
知识点:风避人是通过空调摄像头，计算出人体实时位置，让风避开人体位置。避风区域可以跟随人体走动而随时变化，人在哪里，避风区域就在哪里。
知识点:风避人功能通过空调摄像头计算出人体实时位置，避风区域会跟随人体走动而变化，即人在哪里，避风区域就在哪里。
知识点:风避人通过空调摄像头计算出人体实时位置，然后让风避开人体位置。避风区域能随人体走动而变化，人在哪，避风区域就在哪。
问题:智慧知冷暖的运行方式是怎样的的答案是智慧知冷暖通过红外传感器检测房间空气温度和人体表温度，结合房间空气湿度和空调送风状态等参数，计算人体冷热感。然后根据这

/tmp/ipykernel_3332676/1086294788.py:191: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  model_inputs["position_ids"] = torch.unsqueeze(torch.range(0, candidate_input_ids.shape[1]-1, 1, dtype=torch.int64), dim=0)[..., min(-candidate_length-1, -1):]


知冷暖通过
红外传感器检测房间空气温度和人体表温度，
结合
房间空气湿
度和空调送风状态等参数，计算人体冷
热感。然后
根据这个
冷热感，自动调整空调运行参数，使得
房间温度
达到人体感到舒适的状态，无需人手动遥控
。

共耗时=41.223499059677124 
*************************************************************************************


### 调用chatglm原流失生成函数 （每一行表示一次推理结果）

In [21]:
for p in prompts:
    inputs = tokenizer(p, return_tensors="pt").to(device)
    inputs['max_length'] = 2000
    stream_generate = model.stream_generate(**inputs)
    cur_len = 0
    s_time = time.time()
    for i in stream_generate:
        new_words = tokenizer.decode(i[0][cur_len:].tolist())
        cur_len = len(i[0])
        print(new_words)
        # print(tokenizer.decode(i[0].tolist()))
    print(f'共耗时={(time.time() - s_time)} ')
    print("*************************************************************************************")

[gMASK]sop <system>你是一个AI助手，旨在提供帮助、诚实和无害的对话服务。<user>风吹人，你能解释一下吗<assistant>风吹人是通过空调摄像头，计算出人体实时位置，仅对该区域进行送风。送风区域可以跟随人体走动而随时变化，人动，风也动。</s><user>风吹人功能在空调上要怎么进行操作<assistant>空调的风吹人功能是通过空调摄像头计算出人体的实时位置，然后仅对该区域进行送风。当人体移动时，送风区域也会随之改变，实现人动风也动的效果。</s><user>风避人的原理是什么，怎么做到的<assistant>风避人通过空调摄像头计算出人体实时位置，让风避开人体位置。避风区域能随人体走动而变化，人在哪，避风区域就在哪。</s><user>可供参考的知识库有：
问题:智慧知冷暖是什么的答案是智慧知冷暖是在制冷和制热模式下，红外传感器在有效检测范围内检测房间空气温度和人体表温度，空调结合房间空气湿度、空调送风状态等参数，计算人体冷热感。空调根据冷热感，自动调整运行参数，不用人手动遥控空调，让房间空气温度达到让人体感到舒适的状态。
问题:智慧知冷暖的运行参数是如何自动调整的的答案是智慧知冷暖通过红外传感器检测房间空气温度和人体表温度，结合房间空气湿度、空调送风状态等参数，计算人体冷热感。然后根据这些数据，自动调整运行参数，使得房间空气温度达到让人体感到舒适的状态，无需人工手动调整。
知识点:风避人是通过空调摄像头计算出人体实时位置，让风避开人体位置，避风区域可以根据人体的走动而变化。
知识点:空调的风避人功能是通过摄像头计算出人体实时位置，让风避开人体。避风区域可以随着人体走动而变化，实现人在哪，避风区域就在哪的效果。
知识点:风避人是通过空调摄像头，计算出人体实时位置，让风避开人体位置。避风区域可以跟随人体走动而随时变化，人在哪里，避风区域就在哪里。
知识点:风避人功能通过空调摄像头计算出人体实时位置，避风区域会跟随人体走动而变化，即人在哪里，避风区域就在哪里。
知识点:风避人通过空调摄像头计算出人体实时位置，然后让风避开人体位置。避风区域能随人体走动而变化，人在哪，避风区域就在哪。
问题:智慧知冷暖的运行方式是怎样的的答案是智慧知冷暖通过红外传感器检测房间空气温度和人体表温度，结合房间空气湿度和空调送风状态等参数，计算人体冷热感。然后根据这

### 单次测试脚本

In [ ]:
prompt = ""

inputs = tokenizer(prompt, return_tensors="pt").to(device)
inputs['max_length'] = 2000
inputs['is_first_forward'] = True

In [ ]:
stream_generate = model.stream_generate(**inputs)
cur_len = 0
s_time = time.time()
for i in stream_generate:
    new_words = tokenizer.decode(i[0][cur_len:].tolist())
    cur_len = len(i[0])
    print(new_words)
    print(f'单次流式耗时={(time.time() - s_time)} ')
# print(tokenizer.decode(i[0].tolist()))

In [ ]:
stream_generate_pld = model.stream_generate_assisted_by_prompt_lookup_decoding(**inputs)

cur_len = 0
s_time = time.time()
for i in stream_generate_pld:
    new_words = tokenizer.decode(i[0][cur_len:].tolist())
    cur_len = len(i[0])
    print(new_words)
    print(f'单次流式耗时={(time.time() - s_time)} ')
    # print(tokenizer.decode(i[0].tolist()))